## step-1 Read files from BLOB to Databricks

In [0]:
storage_account_name = "adfpracticeces"
storage_account_key = ""
container_name = "cricbuzz-bronze"

# Set the Azure Storage account key in Spark configuration
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

# Load data from Azure Blob Storage
df = spark.read.text(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/")

# # Show the data
# df.display()


## Step-2 File Reading - Directly from Blob

In [0]:
df_full_data=spark.read.option("header",True).option("Inferschema",True).option("delimiter",",").json(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/cricbuzz-bronze/full_data1.json")

# df_live=spark.read.option("header",True).option("Inferschema",True).option("delimiter",",").json(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/live_matches.json")

# df_result=spark.read.option("header",True).option("Inferschema",True).option("delimiter",",").json(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/result_matches.json")

## Seperating the Live,Result and Fixture Data

In [0]:
from pyspark.sql.functions import col

# Initialize empty lists to store data
existing_live_matches = []
existing_fixture_matches = []
existing_result_matches = []

# Separate the data in the DataFrame based on the "ms" field
new_live_matches = df_full_data.filter(col("ms") == "live")
new_fixture_matches = df_full_data.filter(col("ms") == "fixture")
new_result_matches = df_full_data.filter(col("ms") == "result")

# Append the new data to the existing data
existing_live_matches.extend(new_live_matches.collect())
existing_fixture_matches.extend(new_fixture_matches.collect())
existing_result_matches.extend(new_result_matches.collect())

# Now, you can use these lists as needed or upload them to your blob storage.


## Transformations

In [0]:

from pyspark.sql.functions import col,split
from pyspark.sql.functions import col, regexp_replace
 

# # Check if 'fixture' is in the 'ms' column of df_fixture
if new_fixture_matches.filter(col("ms").contains("fixture")).count() > 0:
    new_fixture_matches = new_fixture_matches.drop("t2img", "t1img")
    new_fixture_matches=new_fixture_matches.withColumnRenamed("t1", "Team_A")\
        .withColumnRenamed("t2","Team_B")\
        .withColumnRenamed("t1s","Team_1_Score")\
        .withColumnRenamed("t2s","Team_2_Score")\
        .withColumnRenamed("ms","Match_Status")\
        .withColumnRenamed("matchType","Match_Type")
    ## Removing the Code names of the teams
    pattern = r'\s*\[.*\]'
    new_fixture_matches = new_fixture_matches.withColumn("Team_A", regexp_replace(col("Team_A"), pattern, "")).withColumn('Team_B',regexp_replace(col("Team_B"),pattern, ""))
    new_fixture_matches=new_fixture_matches.drop(col('Team_Code'))
    
    # Split the "dateTimeGMT" column into separate "Date" and "Time" columns
    new_fixture_matches = new_fixture_matches.withColumn("Date", split(col("dateTimeGMT"), "T")[0])
    new_fixture_matches = new_fixture_matches.withColumn("Time", split(col("dateTimeGMT"), "T")[1])
    new_fixture_matches=new_fixture_matches.drop('dateTimeGMT')
    new_fixture_matches.display(truncate=False)

else:
    print("All rows contains Live greater than 0")

if new_live_matches.filter(col("ms").contains("live")).count() > 0:
    new_live_matches = new_live_matches.drop("t2img", "t1img")
    new_live_matches=new_live_matches.withColumnRenamed("t1", "Team_A")\
        .withColumnRenamed("t2","Team_B")\
        .withColumnRenamed("t1s","Team_1_Score")\
        .withColumnRenamed("t2s","Team_2_Score")\
        .withColumnRenamed("ms","Match_Status")\
        .withColumnRenamed("matchType","Match_Type")
    ## Removing the Code names of the teams
    pattern = r'\s*\[.*\]'
    new_live_matches = new_live_matches.withColumn("Team_A", regexp_replace(col("Team_A"), pattern, "")).withColumn('Team_B',regexp_replace(col("Team_B"),pattern, ""))
    new_live_matches=new_live_matches.drop(col('Team_Code'))
    
    # Split the "dateTimeGMT" column into separate "Date" and "Time" columns
    new_live_matches = new_live_matches.withColumn("Date", split(col("dateTimeGMT"), "T")[0])
    new_live_matches = new_live_matches.withColumn("Time", split(col("dateTimeGMT"), "T")[1])
    new_live_matches=new_live_matches.drop('dateTimeGMT')
    new_live_matches.display(truncate=False)
else:
    print("All rows contains Live greater than 0")

if new_result_matches.filter(col('ms').contains('result')).count()>0:
    new_result_matches = new_result_matches.drop("t2img", "t1img")    
    new_result_matches = new_result_matches.withColumnRenamed("t1", "Team_A")\
        .withColumnRenamed("t2", "Team_B")\
        .withColumnRenamed("t1s", "Team_1_Score")\
        .withColumnRenamed("t2s", "Team_2_Score")\
        .withColumnRenamed("ms", "Match_Status")\
        .withColumnRenamed("matchType", "Match_Type")

    ## Removing the Code names of the teams
    pattern = r'\s*\[.*\]'
    new_result_matches = new_result_matches.withColumn("Team_A", regexp_replace(col("Team_A"), pattern, ""))\
                        .withColumn('Team_B',regexp_replace(col("Team_B"),pattern, ""))
    new_result_matches = new_result_matches.drop(col('Team_Code'))
    
    # Split the "dateTimeGMT" column into separate "Date" and "Time" columns
    new_result_matches = new_result_matches.withColumn("Date", split(col("dateTimeGMT"), "T")[0])
    new_result_matches = new_result_matches.withColumn("Time", split(col("dateTimeGMT"), "T")[1])
    new_result_matches=new_result_matches.drop('dateTimeGMT')
    new_result_matches.display(truncate=False)
else:
    print("All rows contains Live greater than 0")

   


id,Match_Type,Match_Status,status,Team_A,Team_1_Score,Team_B,Team_2_Score,Date,Time
1a97a643-b30c-4d6a-9858-8060d7377d48,t20,fixture,Match not started,Nepal,,Singapore,,2023-10-30,03:15:00
c0debcc8-80e1-4258-96df-49d430adf89f,t20,fixture,Match not started,Bahrain,,United Arab Emirates,,2023-10-30,03:15:00
5b4c9819-990f-4332-b8d3-4779e4404072,t20,fixture,Match not started,Bangladesh Women,,Pakistan Women,,2023-10-29,10:30:00
ce6b7e52-c948-4252-9b47-fced87202add,t20,fixture,Match not started,Namibia,,Zimbabwe,,2023-10-29,09:00:00
7302a459-d31b-4629-9ac3-e8c9eefc9fb4,odi,fixture,Match not started,England,,India,,2023-10-29,08:30:00
a4c0a1c2-574f-4dad-9f4f-677d91bc74e4,t20,fixture,Match not started,Adelaide Strikers Women,,Brisbane Heat Women,,2023-10-29,08:10:00
684439ea-47e2-4391-b0c8-0f7a43e7ef10,t20,fixture,Match not started,Melbourne Stars Women,,Perth Scorchers Women,,2023-10-29,04:00:00
1d6dea8b-f1d6-49f5-895a-729c773c074c,odi,fixture,Match not started,Bangladesh,,Netherlands,,2023-10-28,08:30:00
6ba8357e-3182-4008-9625-8962bd4e08cb,odi,fixture,Match not started,Australia,,New Zealand,,2023-10-28,05:00:00
f8d4ffe9-16ad-46c0-a3ca-f8d007e6f5fa,t20,fixture,Match not started,Melbourne Renegades Women,,Sydney Sixers Women,,2023-10-28,01:00:00


id,Match_Type,Match_Status,status,Team_A,Team_1_Score,Team_B,Team_2_Score,Date,Time
d48c1e8e-ec2e-4381-a100-c46e16104835,t20,live,Mizoram opt to bowl,Haryana,48/2 (8),Mizoram,,2023-10-23,05:30:00
4496c58b-47b2-4805-983d-df060fc485c7,t20,live,Match not started,Karnataka,,Nagaland,,2023-10-23,05:30:00
71dfe8c5-eefe-49b9-ab8d-9ccec2e24eed,t20,live,Tripura opt to bowl,Tripura,,Uttar Pradesh,59/2 (8.4),2023-10-23,05:30:00
60520918-1033-4511-9690-bcf4af29b3fb,t20,live,Match not started,Puducherry,,Vidarbha,,2023-10-23,05:30:00
ed43b088-4225-4c0e-812e-804cecf895df,t20,live,Maharashtra opt to bowl,Jharkhand,43/1 (5),Maharashtra,,2023-10-23,05:30:00
fa812050-6aee-4f0c-8444-70ba9d148bd8,t20,live,Saurashtra opt to bowl,Arunachal Pradesh,20/6 (11),Saurashtra,,2023-10-23,05:30:00
74a59b0b-3cac-4de2-85a0-dcce6888fd2b,t20,live,Match not started,Kerala,,Sikkim,,2023-10-23,05:30:00
8aa7165e-369c-49db-a04e-9f01728a83e5,t20,live,Nepal opt to bowl,Nepal,,United Arab Emirates,80/3 (11),2023-10-23,05:30:00
7acee0b5-cdc3-4189-a61b-f5dec9c07c55,t20,live,Adelaide Strikers Women need 99 runs in 51 balls,Adelaide Strikers Women,69/4 (11.3),Melbourne Renegades Women,167/3 (20),2023-10-23,04:00:00
3aa10c7b-07ed-4b56-9e97-fe37963c3a88,t20,live,Baroda opt to bowl,Baroda,,Hyderabad,79/3 (10),2023-10-23,03:30:00


id,Match_Type,Match_Status,status,Team_A,Team_1_Score,Team_B,Team_2_Score,Date,Time
0f3b914b-a85c-4941-bdd7-64b65da9fa23,odi,result,India won by 4 wkts,India,274/6 (48),New Zealand,273/10 (50),2023-10-22,08:30:00
efe56454-a41f-43cf-8229-898231479b6b,odi,result,Western Province won by 5 wkts,Boland,255/10 (50),Western Province,260/5 (32.5),2023-10-22,08:00:00
636e2ebe-838e-4922-a5bb-4c2da0692bbc,odi,result,North West won by 7 wkts,North West,97/3 (20.4),Warriors,94/10 (34.5),2023-10-22,08:00:00
f7add24a-3b32-4b78-a40d-87b4c816a773,odi,result,Lions won by 8 wkts,Lions,79/2 (11.1),Titans,78/10 (22.5),2023-10-22,08:00:00
b3f57858-1d3a-42d7-b5b1-44bbc78885bc,odi,result,There is no scorecard available for this match.,Dolphins,,KwaZulu-Natal Inland,,2023-10-22,07:30:00
a2af1889-9654-4355-8937-7c2f21273775,t20,result,Sydney Thunder Women won by 42 runs,Sydney Sixers Women,148/9 (20),Sydney Thunder Women,190/5 (20),2023-10-22,06:25:00
229be150-ee2c-4929-983a-1a4692dc31ba,t20,result,United Arab Emirates won by 6 wkts,Hong Kong,149/5 (20),United Arab Emirates,153/4 (18),2023-10-22,05:30:00
000ce88e-2b99-44af-bf14-fd6822c014f1,t20,result,Brisbane Heat Women won by 50 runs,Brisbane Heat Women,229/7 (20),Perth Scorchers Women,179/8 (20),2023-10-22,02:40:00
f807e81a-9d1f-48e9-8ed2-afcbc99fc3f3,t20,result,Hyderabad won by 6 wkts,Hyderabad,115/4 (15.1),Mizoram,114/10 (20),2023-10-21,11:00:00
51e1ca52-e2f0-4b9c-92b4-467c34166835,t20,result,Uttar Pradesh won by 87 runs,Nagaland,100/7 (20),Uttar Pradesh,187/6 (20),2023-10-21,11:00:00


### Spliting the score column of live matches

In [0]:
    new_live_matches = new_live_matches.withColumn("Team_A_Score", split(col("Team_1_Score"), "/")[0])
    new_live_matches = new_live_matches.withColumn("Team_B_Score", split(col("Team_2_Score"), "/")[0])
    new_live_matches = new_live_matches.withColumn("Team_A_Wickets", split(col("Team_1_Score"), "/")[1])
    new_live_matches = new_live_matches.withColumn("Team_B_Wickets", split(col("Team_2_Score"), "/")[1])
    new_live_matches = new_live_matches.withColumn("Team_A_overs", split(col("Team_A_Wickets"), " ")[1])
    new_live_matches = new_live_matches.withColumn("Team_B_overs", split(col("Team_B_Wickets"), " ")[1])
    new_live_matches=new_live_matches.drop('Team_1_Score')
    new_live_matches=new_live_matches.drop('Team_2_Score')
    pattern = r'\s*\(\d+(\.\d+)?\)'
    new_live_matches = new_live_matches.withColumn("Team_A_Wickets", regexp_replace(col("Team_A_Wickets"), pattern, "")).withColumn("Team_B_Wickets", regexp_replace(col("Team_B_Wickets"), pattern, ""))
    
    new_live_matches.display(truncate=False)

id,Match_Type,Match_Status,status,Team_A,Team_B,Date,Time,Team_A_Score,Team_B_Score,Team_A_Wickets,Team_B_Wickets,Team_A_overs,Team_B_overs
d48c1e8e-ec2e-4381-a100-c46e16104835,t20,live,Mizoram opt to bowl,Haryana,Mizoram,2023-10-23,05:30:00,48,,2,null,(8),null
4496c58b-47b2-4805-983d-df060fc485c7,t20,live,Match not started,Karnataka,Nagaland,2023-10-23,05:30:00,,,null,null,null,null
71dfe8c5-eefe-49b9-ab8d-9ccec2e24eed,t20,live,Tripura opt to bowl,Tripura,Uttar Pradesh,2023-10-23,05:30:00,,59,null,2,null,(8.4)
60520918-1033-4511-9690-bcf4af29b3fb,t20,live,Match not started,Puducherry,Vidarbha,2023-10-23,05:30:00,,,null,null,null,null
ed43b088-4225-4c0e-812e-804cecf895df,t20,live,Maharashtra opt to bowl,Jharkhand,Maharashtra,2023-10-23,05:30:00,43,,1,null,(5),null
fa812050-6aee-4f0c-8444-70ba9d148bd8,t20,live,Saurashtra opt to bowl,Arunachal Pradesh,Saurashtra,2023-10-23,05:30:00,20,,6,null,(11),null
74a59b0b-3cac-4de2-85a0-dcce6888fd2b,t20,live,Match not started,Kerala,Sikkim,2023-10-23,05:30:00,,,null,null,null,null
8aa7165e-369c-49db-a04e-9f01728a83e5,t20,live,Nepal opt to bowl,Nepal,United Arab Emirates,2023-10-23,05:30:00,,80,null,3,null,(11)
7acee0b5-cdc3-4189-a61b-f5dec9c07c55,t20,live,Adelaide Strikers Women need 99 runs in 51 balls,Adelaide Strikers Women,Melbourne Renegades Women,2023-10-23,04:00:00,69,167,4,3,(11.3),(20)
3aa10c7b-07ed-4b56-9e97-fe37963c3a88,t20,live,Baroda opt to bowl,Baroda,Hyderabad,2023-10-23,03:30:00,,79,null,3,null,(10)


### Spliting the score column of fixture matches

In [0]:
    new_fixture_matches = new_fixture_matches.withColumn("Team_A_Score", split(col("Team_1_Score"), "/")[0])
    new_fixture_matches = new_fixture_matches.withColumn("Team_B_Score", split(col("Team_2_Score"), "/")[0])
    new_fixture_matches = new_fixture_matches.withColumn("Team_A_Wickets", split(col("Team_1_Score"), "/")[1])
    new_fixture_matches = new_fixture_matches.withColumn("Team_B_Wickets", split(col("Team_2_Score"), "/")[1])
    new_fixture_matches = new_fixture_matches.withColumn("Team_A_overs", split(col("Team_A_Wickets"), " ")[1])
    new_fixture_matches = new_fixture_matches.withColumn("Team_B_overs", split(col("Team_B_Wickets"), " ")[1])
    new_fixture_matches=new_fixture_matches.drop('Team_1_Score')
    new_fixture_matches=new_fixture_matches.drop('Team_2_Score')
    pattern = r'\s*\(\d+(\.\d+)?\)'
    new_fixture_matches = new_fixture_matches.withColumn("Team_A_Wickets", regexp_replace(col("Team_A_Wickets"), pattern, "")).withColumn("Team_B_Wickets", regexp_replace(col("Team_B_Wickets"), pattern, ""))
    
    new_fixture_matches.display(truncate=False)

id,Match_Type,Match_Status,status,Team_A,Team_B,Date,Time,Team_A_Score,Team_B_Score,Team_A_Wickets,Team_B_Wickets,Team_A_overs,Team_B_overs
1a97a643-b30c-4d6a-9858-8060d7377d48,t20,fixture,Match not started,Nepal,Singapore,2023-10-30,03:15:00,,,null,null,null,null
c0debcc8-80e1-4258-96df-49d430adf89f,t20,fixture,Match not started,Bahrain,United Arab Emirates,2023-10-30,03:15:00,,,null,null,null,null
5b4c9819-990f-4332-b8d3-4779e4404072,t20,fixture,Match not started,Bangladesh Women,Pakistan Women,2023-10-29,10:30:00,,,null,null,null,null
ce6b7e52-c948-4252-9b47-fced87202add,t20,fixture,Match not started,Namibia,Zimbabwe,2023-10-29,09:00:00,,,null,null,null,null
7302a459-d31b-4629-9ac3-e8c9eefc9fb4,odi,fixture,Match not started,England,India,2023-10-29,08:30:00,,,null,null,null,null
a4c0a1c2-574f-4dad-9f4f-677d91bc74e4,t20,fixture,Match not started,Adelaide Strikers Women,Brisbane Heat Women,2023-10-29,08:10:00,,,null,null,null,null
684439ea-47e2-4391-b0c8-0f7a43e7ef10,t20,fixture,Match not started,Melbourne Stars Women,Perth Scorchers Women,2023-10-29,04:00:00,,,null,null,null,null
1d6dea8b-f1d6-49f5-895a-729c773c074c,odi,fixture,Match not started,Bangladesh,Netherlands,2023-10-28,08:30:00,,,null,null,null,null
6ba8357e-3182-4008-9625-8962bd4e08cb,odi,fixture,Match not started,Australia,New Zealand,2023-10-28,05:00:00,,,null,null,null,null
f8d4ffe9-16ad-46c0-a3ca-f8d007e6f5fa,t20,fixture,Match not started,Melbourne Renegades Women,Sydney Sixers Women,2023-10-28,01:00:00,,,null,null,null,null


### Spliting the score column of result matches

In [0]:
    new_result_matches = new_result_matches.withColumn("Team_A_Score", split(col("Team_1_Score"), "/")[0])
    new_result_matches = new_result_matches.withColumn("Team_B_Score", split(col("Team_2_Score"), "/")[0])
    new_result_matches = new_result_matches.withColumn("Team_A_Wickets", split(col("Team_1_Score"), "/")[1])
    new_result_matches = new_result_matches.withColumn("Team_B_Wickets", split(col("Team_2_Score"), "/")[1])
    new_result_matches = new_result_matches.withColumn("Team_A_overs", split(col("Team_A_Wickets"), " ")[1])
    new_result_matches = new_result_matches.withColumn("Team_B_overs", split(col("Team_B_Wickets"), " ")[1])
    new_result_matches=new_result_matches.drop('Team_1_Score')
    new_result_matches=new_result_matches.drop('Team_2_Score')
    pattern = r'\s*\(\d+(\.\d+)?\)'
    new_result_matches = new_result_matches.withColumn("Team_A_Wickets", regexp_replace(col("Team_A_Wickets"), pattern, "")).withColumn("Team_B_Wickets", regexp_replace(col("Team_B_Wickets"), pattern, ""))
    
    new_result_matches.display(truncate=False)

id,Match_Type,Match_Status,status,Team_A,Team_B,Date,Time,Team_A_Score,Team_B_Score,Team_A_Wickets,Team_B_Wickets,Team_A_overs,Team_B_overs
0f3b914b-a85c-4941-bdd7-64b65da9fa23,odi,result,India won by 4 wkts,India,New Zealand,2023-10-22,08:30:00,274,273,6,10,(48),(50)
efe56454-a41f-43cf-8229-898231479b6b,odi,result,Western Province won by 5 wkts,Boland,Western Province,2023-10-22,08:00:00,255,260,10,5,(50),(32.5)
636e2ebe-838e-4922-a5bb-4c2da0692bbc,odi,result,North West won by 7 wkts,North West,Warriors,2023-10-22,08:00:00,97,94,3,10,(20.4),(34.5)
f7add24a-3b32-4b78-a40d-87b4c816a773,odi,result,Lions won by 8 wkts,Lions,Titans,2023-10-22,08:00:00,79,78,2,10,(11.1),(22.5)
b3f57858-1d3a-42d7-b5b1-44bbc78885bc,odi,result,There is no scorecard available for this match.,Dolphins,KwaZulu-Natal Inland,2023-10-22,07:30:00,,,null,null,null,null
a2af1889-9654-4355-8937-7c2f21273775,t20,result,Sydney Thunder Women won by 42 runs,Sydney Sixers Women,Sydney Thunder Women,2023-10-22,06:25:00,148,190,9,5,(20),(20)
229be150-ee2c-4929-983a-1a4692dc31ba,t20,result,United Arab Emirates won by 6 wkts,Hong Kong,United Arab Emirates,2023-10-22,05:30:00,149,153,5,4,(20),(18)
000ce88e-2b99-44af-bf14-fd6822c014f1,t20,result,Brisbane Heat Women won by 50 runs,Brisbane Heat Women,Perth Scorchers Women,2023-10-22,02:40:00,229,179,7,8,(20),(20)
f807e81a-9d1f-48e9-8ed2-afcbc99fc3f3,t20,result,Hyderabad won by 6 wkts,Hyderabad,Mizoram,2023-10-21,11:00:00,115,114,4,10,(15.1),(20)
51e1ca52-e2f0-4b9c-92b4-467c34166835,t20,result,Uttar Pradesh won by 87 runs,Nagaland,Uttar Pradesh,2023-10-21,11:00:00,100,187,7,6,(20),(20)


## Creating the DataBase in Hive Meta Store

In [0]:
# spark.sql("CREATE DATABASE IF NOT EXISTS cricbuzz")


DataFrame[]

## Storing Dataframes into tables

In [0]:
# new_result_matches.createOrReplaceTempView("result_table")
# new_result_matches.write.saveAsTable("cricbuzz.result_table")
new_result_matches.write.mode("append").saveAsTable("cricbuzz.result_table")



In [0]:
# new_live_matches.createOrReplaceTempView("live_table")
new_live_matches.write.mode("overwrite").saveAsTable("cricbuzz.live_table")

In [0]:
%sql
select * from cricbuzz.result_table

id,Match_Type,Match_Status,status,Team_A,Team_B,Date,Time,Team_A_Score,Team_B_Score,Team_A_Wickets,Team_B_Wickets,Team_A_overs,Team_B_overs
8ee9059e-d9d0-4116-925e-2e82826be05f,t20,result,Baroda won by 3 wkts,Baroda,Mumbai,2023-11-02,03:30:00,149,148,7,8,(18.5),(20)
4be804f3-5e60-404e-a0da-250b7355c995,t20,result,Oman won by 5 runs,Nepal,Oman,2023-11-02,03:15:00,140,145,10,9,(20),(20)
86850df6-c372-4079-9051-f08ac001f7b2,t20,result,United Arab Emirates won by 22 runs,Hong Kong,United Arab Emirates,2023-11-02,03:15:00,154,176,8,6,(20),(20)
143e530a-5b1c-4880-9c86-80e7249368a7,odi,result,South Africa won by 190 runs,New Zealand,South Africa,2023-11-01,08:30:00,167,357,10,4,(35.3),(50)
2b46d9ab-e178-416f-beec-d98fe79482d7,t20,result,Sydney Thunder Women won by 37 runs,Melbourne Renegades Women,Sydney Thunder Women,2023-11-01,04:00:00,153,190,7,5,(20),(20)
c6dea2aa-98ad-4f73-b5f6-a7151c09cba2,t20,result,Assam won by 8 wkts,Assam,Bengal,2023-10-31,11:00:00,142,138,2,8,(17.5),(20)
2e843aec-35c7-4cc4-a25a-3f2e28f30314,t20,result,Perth Scorchers Women won by 36 runs,Perth Scorchers Women,Sydney Sixers Women,2023-10-31,09:10:00,166,130,8,9,(20),(20)
96a377ab-cb46-4b7a-83c0-34721a790935,odi,result,Pakistan won by 7 wkts,Bangladesh,Pakistan,2023-10-31,08:30:00,204,205,10,3,(45.1),(32.3)
4f8503dd-edea-4f91-bf4c-174c6bc044c4,t20,result,United Arab Emirates won by 5 wkts,Kuwait,United Arab Emirates,2023-10-31,07:45:00,88,94,8,5,(20),(14.2)
5b5b2498-97b4-4b24-ba96-57790b2b14f7,t20,result,Nepal won by 6 wkts,Malaysia,Nepal,2023-10-31,07:45:00,165,166,9,4,(20),(18)


In [0]:
# new_fixture_matches.createOrReplaceTempView("fixture_table")
new_fixture_matches.write.mode("overwrite").saveAsTable("cricbuzz.fixture_table")